In [2]:
import org.apache.spark.sql.{DataFrame, Dataset, Row, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
val spark = SparkSession.builder.master("local[*]").appName("dataProcessing").getOrCreate()
import spark.implicits._

spark = org.apache.spark.sql.SparkSession@3d6a2973


org.apache.spark.sql.SparkSession@3d6a2973

In [34]:
import java.io.File
import scala.io.Source

val path = System.getProperty("user.dir")
val seasons = 1990 to 2018
val teams = 1 to 30

for (i <- seasons){
    for (j <- teams){
        val file = path + "/season_averages/" + i + "/" + j + ".json"
        try{
            var df_stats = spark.read.json(file)
            var elem = df_stats.select(explode($"data").as("data"))
            var minutes = elem.select("data.min")
            val split_min = elem.withColumn("seconds", split($"data.min", ":")(0) * 60 + split($"data.min", ":")(1))
            val best_12 = split_min.orderBy(desc("seconds")).limit(12)
            val best_12_struct = best_12.withColumn("row", struct(col("data.ast"), col("data.blk"), col("data.dreb"),
                                                                  col("data.fg3_pct"), col("data.fg3a"), 
                                                                  col("data.fg3m"), col("data.fg_pct"), 
                                                                  col("data.fga"), col("data.fgm"), 
                                                                  col("data.ft_pct"), col("data.fta"),
                                                                  col("data.ftm"), col("data.games_played"), 
                                                                  col("seconds"), col("data.oreb"), col("data.pf"),
                                                                  col("data.player_id"), 
                                                                  col("data.pts"), col("data.reb"), 
                                                                  col("data.season"), col("data.stl"), 
                                                                  col("data.turnover"))).agg(collect_list(col("row")).as("data"))
            
            val best_12_data = best_12_struct.select("data")
            best_12_data.write.mode("overwrite").format("json").save(path + "/season_averages/" + i + "/" + j)


            val best_12_36 = best_12.withColumn("ast", (col("data.ast") / col("seconds") * 2160))
                                    .withColumn("blk", col("data.blk") / col("seconds") * 2160)
                                    .withColumn("dreb", col("data.dreb") / col("seconds") * 2160)
                                    .withColumn("fg3a", col("data.fg3a") / col("seconds") * 2160)
                                    .withColumn("fg3m", col("data.fg3m") / col("seconds") * 2160)
                                    .withColumn("fga", col("data.fga") / col("seconds") * 2160)
                                    .withColumn("fgm", col("data.fgm") / col("seconds") * 2160)
                                    .withColumn("fta", col("data.fta") / col("seconds") * 2160)
                                    .withColumn("ftm", col("data.ftm") / col("seconds") * 2160)
                                    .withColumn("oreb", col("data.oreb") / col("seconds") * 2160)
                                    .withColumn("pf", col("data.pf") / col("seconds") * 2160)
                                    .withColumn("pts", col("data.pts") / col("seconds") * 2160)
                                    .withColumn("reb", col("data.reb") / col("seconds") * 2160)
                                    .withColumn("seconds", col("seconds") / col("seconds") * 2160)
                                    .withColumn("stl", col("data.stl") / col("seconds") * 2160)
                                    .withColumn("turnover", col("data.turnover") / col("seconds") * 2160)

            val best_12_36_struct = best_12_36.withColumn("row", struct(col("ast"), col("blk"), col("dreb"), 
                                                                        col("data.fg3_pct"), col("fg3a"), 
                                                                        col("fg3m"), col("data.fg_pct"), col("fga"),
                                                                        col("fgm"), col("data.games_played"), 
                                                                        col("seconds"), col("oreb"), 
                                                                        col("pf"), col("data.player_id"),
                                                                        col("pts"), col("reb"), col("data.season"),
                                                                        col("stl"), col("turnover"))).agg(collect_list(col("row")).as("data"))
            val best_12_36_data = best_12_36_struct.select("data")
            best_12_36_data.write.mode("overwrite").format("json").save(path + "/season_averages/" + i + "/" + j + "-36")
            println("File " + file + " saved")
            
        
        }
        catch{
            case e: org.apache.spark.sql.AnalysisException => println("File " + file + " not found")
        }
    } 
}

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/1.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/2.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/3.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/4.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/5.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/6.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/7.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/8.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/9.json saved
File /home/adrian/D

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/16.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/17.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/18.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/19.json not found
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/20.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/21.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/22.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/23.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/24.json saved
File /

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/1.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/2.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/3.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/4.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/5.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/6.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/7.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/8.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1995/9.json saved
File /home/adrian/D

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/16.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/17.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/18.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/19.json not found
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/20.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/21.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/22.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/23.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/24.json saved
File /

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/1.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/2.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/3.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/4.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/5.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/6.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/7.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/8.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2000/9.json saved
File /home/adrian/D

...

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/1.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/2.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/3.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/4.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/5.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/6.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/7.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/8.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2015/9.json saved
File /home/adrian/D

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/16.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/17.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/18.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/19.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/20.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/21.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/22.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/23.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2017/24.json saved
File /home

path = /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames
seasons = Range(1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018)
teams = Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30)


lastException: Throwable = null


Range(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30)